In [1]:
import os
import torch
import json
from transformers import AutoModelForCausalLM, AutoTokenizer
from dotenv import load_dotenv

# Import your RAG class from your rag.py file
from RAG_With_QWEN import RAG

# Method to get the API key from the .env file
def get_api_key(api_name):
    env_path = "../.dummy_env"  # Adjust the path as needed
    load_dotenv(env_path)  # Load the environment variables
    return os.getenv(api_name)

# -------------------------------
# Setup Evaluator Model (Mistral)
# -------------------------------
# Define a custom cache directory
CUSTOM_CACHE_DIR = "/media/volume/vol-VisaWise/models/mistral_cache"

# Load the Mistral model and tokenizer
evaluator_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3",
    cache_dir=CUSTOM_CACHE_DIR,  # Specify custom cache directory
    token=get_api_key('HF_GAURI')
)
device = "cuda" if torch.cuda.is_available() else "cpu"
evaluator_model.to(device)

evaluator_tokenizer = AutoTokenizer.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3",
    cache_dir=CUSTOM_CACHE_DIR,  # Specify the same cache directory
    token=get_api_key('HF_GAURI')
)

# -------------------------------
# Create a QA Dataset for Evaluation
# -------------------------------
qa_dataset = [
    {
        "question": "How can an international student on an F-1 visa maintain their status during graduate studies?",
        "expected_answer": (
            "To maintain F-1 status during graduate studies, an international student must enroll as a full-time student, "
            "make normal progress toward a degree, and comply with all rules and regulations set by their educational "
            "institution and the U.S. government. This includes not engaging in unauthorized employment and keeping "
            "documentation up to date."
        )
    },
    {
        "question": "What is Curricular Practical Training (CPT) and how does it benefit F-1 students?",
        "expected_answer": (
            "CPT is a type of off-campus employment authorization for F-1 students that allows them to gain practical "
            "experience in their field of study. It is usually offered as part of an internship, co-op, or practicum "
            "course, and must be an integral part of the student's curriculum."
        )
    },
    {
        "question": "What are the eligibility criteria for Optional Practical Training (OPT) for international students on an F-1 visa?",
        "expected_answer": (
            "OPT allows F-1 students to work in the United States for up to 12 months after completing their degree. "
            "Eligibility typically requires that the student has maintained valid F-1 status, completed at least one "
            "academic year of study, and that the employment is directly related to the student's major field of study."
        )
    }
]

# -------------------------------
# Define Evaluation Function
# -------------------------------
def evaluate_answer(question, generated_answer, expected_answer):
    eval_prompt = (
        f"Question: {question}\n\n"
        f"Expected Answer: {expected_answer}\n\n"
        f"Generated Answer: {generated_answer}\n\n"
        "Please evaluate the generated answer compared to the expected answer. Provide a score between 1 (poor) "
        "and 5 (excellent) along with a brief justification for your score.\n\n"
        "Score and Justification:"
    )
    
    tokens = evaluator_tokenizer.encode(eval_prompt, return_tensors="pt").to(device)
    generated_ids = evaluator_model.generate(tokens, max_new_tokens=100, do_sample=True)
    evaluation_text = evaluator_tokenizer.decode(generated_ids[0].tolist(), skip_special_tokens=True)
    return evaluation_text.strip()

# -------------------------------
# Main Evaluation Routine
# -------------------------------
def main():
    rag_model = RAG()
    evaluations = []  
    
    for qa in qa_dataset:
        question = qa["question"]
        expected_answer = qa["expected_answer"]
        print(f"\nEvaluating Question: {question}")
        
        generated_answer = rag_model.generate_answer(question)
        print("Generated Answer:", generated_answer)
        
        eval_result = evaluate_answer(question, generated_answer, expected_answer)
        print("Evaluation Result:", eval_result)
        print("-" * 80)
        
        evaluations.append({
            "question": question,
            "expected_answer": expected_answer,
            "generated_answer": generated_answer,
            "evaluation": eval_result
        })
    
    with open("rag_evaluation_results.json", "w") as outfile:
        json.dump(evaluations, outfile, indent=4)
    
    print("\nEvaluation completed. Results saved to rag_evaluation_results.json.")

if __name__ == "__main__":
    main()


config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

...

Device set to use cpu



Evaluating Question: How can an international student on an F-1 visa maintain their status during graduate studies?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generated Answer: <|im_sep|>

assistant: To maintain F-1 visa status during graduate studies, an international student must adhere to several key requirements:

1. **Full-time enrollment**: Students must remain enrolled in a full-time course load as determined by their academic institution. This usually means taking the minimum number of credits required for their degree program.

2. **Academic progress**: Maintain satisfactory academic standing. This means staying within the school's standards for grade point average, and completing coursework in a timely manner towards the degree.

3. **Coursework relevance**: Ensure that the courses taken align with the student's field of study. This maintains alignment with the original visa purpose and avoids issues with maintaining status.

4. **On-campus employment**: Follow guidelines for on-campus employment, which includes:
   - Not displacing U.S. citizens or lawful permanent residents (LPRs) from their jobs.
   - Employments must be for the

KeyboardInterrupt: 

# Answer:

Evaluating Question: How can an international student on an F-1 visa maintain their status during graduate studies?

Generated Answer: <|im_sep|>

assistant: To maintain F-1 visa status during graduate studies, an international student must adhere to several key requirements:

1. **Full-time enrollment**: Students must remain enrolled in a full-time course load as determined by their academic institution. This usually means taking the minimum number of credits required for their degree program.

2. **Academic progress**: Maintain satisfactory academic standing. This means staying within the school's standards for grade point average, and completing coursework in a timely manner towards the degree.

3. **Coursework relevance**: Ensure that the courses taken align with the student's field of study. This maintains alignment with the original visa purpose and avoids issues with maintaining status.

4. **On-campus employment**: Follow guidelines for on-campus employment, which includes:
   - Not displacing U.S. citizens or lawful permanent residents (LPRs) from their jobs.
   - Employments must be for the school or for an educationally affiliated organization. Examples include school
Evaluation Result: Question: How can an international student on an F-1 visa maintain their status during graduate studies?

Expected Answer: To maintain F-1 status during graduate studies, an international student must enroll as a full-time student, make normal progress toward a degree, and comply with all rules and regulations set by their educational institution and the U.S. government. This includes not engaging in unauthorized employment and keeping documentation up to date.

Generated Answer: <|im_sep|>

assistant: To maintain F-1 visa status during graduate studies, an international student must adhere to several key requirements:

1. **Full-time enrollment**: Students must remain enrolled in a full-time course load as determined by their academic institution. This usually means taking the minimum number of credits required for their degree program.

2. **Academic progress**: Maintain satisfactory academic standing. This means staying within the school's standards for grade point average, and completing coursework in a timely manner towards the degree.

3. **Coursework relevance**: Ensure that the courses taken align with the student's field of study. This maintains alignment with the original visa purpose and avoids issues with maintaining status.

4. **On-campus employment**: Follow guidelines for on-campus employment, which includes:
   - Not displacing U.S. citizens or lawful permanent residents (LPRs) from their jobs.
   - Employments must be for the school or for an educationally affiliated organization. Examples include school

Please evaluate the generated answer compared to the expected answer. Provide a score between 1 (poor) and 5 (excellent) along with a brief justification for your score.

Score and Justification: Excellent 5/5
The generated answer is comprehensive and answers the question accurately by listing out the requirements for maintaining F-1 visa status during graduate studies while providing additional context on each requirement.

- It clearly highlights the importance of full-time enrollment, maintaining satisfactory academic standing, ensuring coursework relevance, and following guidelines for on-campus employment.
- The answer also reinforces the need for alignment with the original visa purpose to avoid issues with maintaining status
--------------------------------------------------------------------------------

Evaluating Question: What is Curricular Practical Training (CPT) and how does it benefit F-1 students?


# Half precision

In [1]:
import os
import torch
import json
from transformers import AutoModelForCausalLM, AutoTokenizer
from dotenv import load_dotenv

# Import your RAG class from your rag.py file
from RAG_With_QWEN import RAG

# Method to get the API key from the .env file
def get_api_key(api_name):
    env_path = "../.dummy_env"  # Adjust the path as needed
    load_dotenv(env_path)  # Load the environment variables
    return os.getenv(api_name)

# -------------------------------
# Setup Evaluator Model (Mistral)
# -------------------------------
# Define a custom cache directory
CUSTOM_CACHE_DIR = "/media/volume/vol-VisaWise/models/mistral_cache"

# Load the Mistral model and tokenizer in FP16 for faster inference
evaluator_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3",
    cache_dir=CUSTOM_CACHE_DIR,  # Specify custom cache directory
    torch_dtype=torch.float16,   # Load in FP16
    token=get_api_key('HF_GAURI')
)
device = "cuda" if torch.cuda.is_available() else "cpu"
evaluator_model.to(device)
evaluator_model.eval()  # Set model to evaluation mode

evaluator_tokenizer = AutoTokenizer.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3",
    cache_dir=CUSTOM_CACHE_DIR,  # Specify the same cache directory
    token=get_api_key('HF_GAURI')
)

# -------------------------------
# Create a QA Dataset for Evaluation
# -------------------------------
qa_dataset = [
    {
        "question": "How can an international student on an F-1 visa maintain their status during graduate studies?",
        "expected_answer": (
            "To maintain F-1 status during graduate studies, an international student must enroll as a full-time student, "
            "make normal progress toward a degree, and comply with all rules and regulations set by their educational "
            "institution and the U.S. government. This includes not engaging in unauthorized employment and keeping "
            "documentation up to date."
        )
    },
    {
        "question": "What is Curricular Practical Training (CPT) and how does it benefit F-1 students?",
        "expected_answer": (
            "CPT is a type of off-campus employment authorization for F-1 students that allows them to gain practical "
            "experience in their field of study. It is usually offered as part of an internship, co-op, or practicum "
            "course, and must be an integral part of the student's curriculum."
        )
    },
    {
        "question": "What are the eligibility criteria for Optional Practical Training (OPT) for international students on an F-1 visa?",
        "expected_answer": (
            "OPT allows F-1 students to work in the United States for up to 12 months after completing their degree. "
            "Eligibility typically requires that the student has maintained valid F-1 status, completed at least one "
            "academic year of study, and that the employment is directly related to the student's major field of study."
        )
    }
]

# -------------------------------
# Define Evaluation Function
# -------------------------------
def evaluate_answer(question, generated_answer, expected_answer):
    eval_prompt = (
        f"Question: {question}\n\n"
        f"Expected Answer: {expected_answer}\n\n"
        f"Generated Answer: {generated_answer}\n\n"
        "Please evaluate the generated answer compared to the expected answer. Provide a score between 1 (poor) "
        "and 5 (excellent) along with a brief justification for your score.\n\n"
        "Score and Justification:"
    )
    
    # Tokenize and move tokens to GPU
    tokens = evaluator_tokenizer.encode(eval_prompt, return_tensors="pt").to(device)
    
    # Disable gradient calculations and use AMP for efficiency
    with torch.no_grad():
        with torch.cuda.amp.autocast():
            generated_ids = evaluator_model.generate(tokens, max_new_tokens=100, do_sample=True)
    
    evaluation_text = evaluator_tokenizer.decode(generated_ids[0].tolist(), skip_special_tokens=True)
    return evaluation_text.strip()

# -------------------------------
# Main Evaluation Routine
# -------------------------------
def main():
    rag_model = RAG()
    evaluations = []  
    
    for qa in qa_dataset:
        question = qa["question"]
        expected_answer = qa["expected_answer"]
        print(f"\nEvaluating Question: {question}")
        
        # Assuming rag_model also uses GPU, make sure to optimize it similarly if possible
        generated_answer = rag_model.generate_answer(question)
        print("Generated Answer:", generated_answer)
        
        eval_result = evaluate_answer(question, generated_answer, expected_answer)
        print("Evaluation Result:", eval_result)
        print("-" * 80)
        
        evaluations.append({
            "question": question,
            "expected_answer": expected_answer,
            "generated_answer": generated_answer,
            "evaluation": eval_result
        })
    
    with open("rag_evaluation_results.json", "w") as outfile:
        json.dump(evaluations, outfile, indent=4)
    
    print("\nEvaluation completed. Results saved to rag_evaluation_results.json.")

if __name__ == "__main__":
    main()


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


Using device: cuda

Evaluating Question: How can an international student on an F-1 visa maintain their status during graduate studies?


...

Generated Answer: <|im_sep|>assistant: An international student on an F-1 visa maintains their status during graduate studies by adhering to several key requirements:

1. **Full-time enrollment**: Continue enrolling in a full-time course load for each term/semester. The exact number of credit hours required varies depending on the institution's policies, but typically it's around 12 credit hours or more.

2. **Course progression**: Make consistent progress towards completing their degree. This usually means successfully passing courses and fulfilling all academic requirements for graduation.

3. **Engage with academic advisors**: Regularly meet with academic advisors to ensure that the chosen courses align with the degree requirements and that the academic plan is on track.

4. **Attend classes regularly**: Active participation in classes is essential. Attendance records are often kept by the institution and can affect visa maintenance.

5. **Maintain a positive academic standing**: Av

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated Answer: assistant: Curricular Practical Training (CPT) is an integral part of an established curriculum, offering alternative work-study, internship, cooperative education, or any required internship or practicum that is part of a cooperative agreement between the school and sponsoring employers. It benefits F-1 students in several ways:

1. **Work Experience**: CPT provides students with practical work experience related to their field of study, enhancing their understanding and skills.

2. **Enhanced Resume**: Participation in CPT can strengthen a student's resume, making them more attractive to potential employers upon graduation.

3. **Networking Opportunities**: Through CPT, students can build professional networks within their industry, which might lead to job opportunities after graduation.

4. **Financial Support**: For some students, CPT offers a source of income to support their studies and living expenses.

5. **Adaptability**: It helps students adapt to the profes

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated Answer: assistant: The eligibility criteria for Optional Practical Training (OPT) for international students on an F-1 visa include:

1. **Eligible Categories**: OPT can be applied for under three categories:
   - (c)(3)(A) - Pre-Completion OPT
   - (c)(3)(B) - Post-Completion OPT
   - (c)(3)(C) - 24-Month Extension of OPT for science, technology, engineering, and mathematics (STEM) students

2. **Duration**: Eligible students can receive up to 12 months of OPT employment authorization before completing their academic studies (pre-completion) and/or after completing their academic studies (post-completion).

3. **STEM Extension**: STEM students may qualify for a 24-month extension of their post-completion OPT.

4. **Academic Status**: Applicants must be in F-1 status for at least one academic year before applying for OPT.

5. **Employment
Evaluation Result: Question: What are the eligibility criteria for Optional Practical Training (OPT) for international students on an F-1 v